# Cvičení 2: Agregace a řazení

In [8]:
import pandas as pd

# Příprava vstupního DataFramu
food_nutrient = pd.read_csv("food_nutrient.csv")
food_sample_100 = pd.read_csv("food_sample_100.csv")
food_other = pd.read_csv("food_other.csv")
food = pd.concat([food_sample_100, food_other], ignore_index=True)
food_merged = pd.merge(food, food_nutrient, on="fdc_id")
branded_food = pd.read_csv("branded_food.csv")
food_merged_brands = pd.merge(food_merged, branded_food, on="fdc_id")

food_merged_brands = food_merged_brands.rename(columns={"name": "nutrient_name"})

## 1. Lipidy a tuky

Podívej se nyní na to, které kategorie potravin obsahují nejvíce lipidů (tuků). 

Nejprve pomocí dotazu vytvoř novou tabulku `food_merged_brands_lipid`, do které pomocí dotazu vlož pouze řádky, které mají jako název výživné látky hodnotu `Total lipid (fat)`. 


In [9]:
food_merged_brands_lipid = food_merged_brands[food_merged_brands["name"] == "Total lipid (fat)"]
food_merged_brands_lipid.shape

(9319, 37)

Poté proveď agregaci podle názvu kategorie a seřaď výslednou tabulku tak, aby nahoře byly vidět kategorie s největším počtem tuků. 

In [10]:
food_merged_brands_lipid_agg = food_merged_brands_lipid.groupby("branded_food_category")["amount"].mean()

# Pro přehlednost zobrazíme pouze prvních deset
# lipid_agg_top10 = food_merged_brands_lipid_agg.sort_values(ascending=False)[:10]
lipid_agg_top10 = food_merged_brands_lipid_agg.sort_values(ascending=False).head(10)

# Bonus: jednodušeji pomocí metody nlargest()
# lipid_agg_top10 = food_merged_brands_lipid_agg.nlargest(10)

lipid_agg_top10

branded_food_category
Oils Edible                                                 93.330000
Vegetable & Cooking Oils                                    78.012391
Butter & Spread                                             66.552667
Butter/Butter Substitutes                                   63.965000
Meat/Poultry/Other Animals – Prepared/Processed             59.520000
Nut & Seed Butters                                          43.891746
Meat/Poultry/Other Animals Sausages – Prepared/Processed    42.860000
Popcorn, Peanuts, Seeds & Related Snacks                    37.204354
Baking Accessories                                          35.400000
Baking/Cooking Mixes (Shelf Stable)                         35.000000
Name: amount, dtype: float64

Porovnej si výslednou tabulku s tabulkou `food_merged_brands_protein_agg`, kterou jsme vytvořili v rámci lekce. 

In [11]:
# Příprava DataFramu food_merged_brands_protein_agg pro cvičení, pokud ho ještě nemáme připravený z lekce
food_merged_brands_protein = food_merged_brands[food_merged_brands["name"] == "Protein"]
food_merged_brands_protein_agg = food_merged_brands_protein.groupby("branded_food_category")["amount"].mean()

Podívej se, zda se některé kategorie objevují v obou tabulkách.

In [12]:
protein_agg_top10 = food_merged_brands_protein_agg.sort_values(ascending=False).head(10)

# Jednodušší alternativa:
# protein_agg_top10 = food_merged_brands_protein_agg.nlargest(10)

protein_agg_top10

branded_food_category
Bacon                                                  45.200000
Green Supplements                                      44.590000
Drinks Flavoured - Ready to Drink                      32.900000
Energy, Protein & Muscle Recovery Drinks               31.831481
Bacon, Sausages & Ribs                                 27.239630
Canned Tuna                                            21.648889
Meat/Poultry/Other Animals  Prepared/Processed         21.522368
Nut & Seed Butters                                     20.651270
Cheese                                                 19.969457
Meat/Poultry/Other Animals – Unprepared/Unprocessed    19.640000
Name: amount, dtype: float64

In [13]:
# Bonus: Výpis kategorií společných pro obě tabulky - např. jako průnik množin
# (protein_agg_top10 je Series s názvy kategorií využitými jako index -> názvy získáme přes .index)
set(protein_agg_top10.index) & set(lipid_agg_top10.index)

{'Nut & Seed Butters'}

## 2. Výrobci a kategorie

Nyní uvažuj, že si chceme udělat přehled o tom, jaký výrobce produkuje jaké typy potravin. 



Proveď agregaci tabulky `food_merged_brands` podle dvou sloupců: `brand_owner` a `branded_food_category`. 

Sloupce musíš metodě `food_merged_brands` zadat jako seznam, tj. musíš použít hranaté závorky. 



In [14]:
food_merged_brands_grouped = food_merged_brands.groupby(["brand_owner", "branded_food_category"])
food_merged_brands_grouped


Dále vyber sloupec `fdc_id` pro provedení agregace a použij agregaci `nunique()`, který vrátí počet unikátních hodnot. 

Nakonec použij metodu `sort_values` s tím, že chceš data seřadit sestupně.

In [15]:
food_merged_brands_grouped["fdc_id"].nunique().sort_values(ascending=False)

brand_owner                branded_food_category               
Ferrara Candy Company      Candy                                   40
CAMPBELL SOUP COMPANY      Prepared Soups                          29
Danone US, LLC             Yogurt                                  24
Bimbo Bakeries USA, Inc.   Breads & Buns                           24
Wells Enterprises, Inc.    Ice Cream & Frozen Yogurt               24
                                                                   ..
Hempler Foods Group LLC    Sausages, Hotdogs & Brats                1
Heinz USA                  Pickles, Olives, Peppers & Relishes      1
Heinz Frozen Food Company  Canned Soup                              1
Heintz & Weber Co Inc      Ketchup, Mustard, BBQ & Cheese Sauce     1
pure batch LLC             Chocolate                                1
Name: fdc_id, Length: 5892, dtype: int64